# <span style="color:lightblue"><center> **CLASSIFICAÇÃO DOS EPISÓDIOS DE MORTE DECORRENTE DA CIRROSE**</center></span>

A seguir, vamos elaborar um processo de classificação com base no dataset [Liver Cirrhosis Stage Classification](https://www.kaggle.com/datasets/aadarshvelu/liver-cirrhosis-stage-classification), com o objetivo de classificar a coluna **"Situação"**. Ou seja, criaremos modelos com Regressão Logística, Árvore de Decisão e Rede Neural, e por último, LGBM (Light Gradient Boosting Machine), para identificar os casos de Sobrevivência, Morte e Transplante dos pacientes de cirrose, vistos na EDA.

Para isso, nós realizamos 3 classificações diferentes:

**1. Morte, Sobrevivência e Transplante**
- Essa primeira classificação contém as 3 classes, não modificamos nada no dataset. O ponto dela é ser capaz de distinguir os 3 casos individualmente.

**2. Mesclando Sobrevivência e Transplante**
- Aqui, nós juntamos as classes *Sobrevivência* e *Transplante* em uma só, com a finalidade de identificar binariamente a situação dos pacientes estudados: ou morto ou vivo.

**3. Ignorando Evento Transplante**
- Por último, fizemos uma classificação deixando de lado a classe *Transplante*, levando em consideração apenas os atributos dos pacientes que sobreviveram que não necessitaram da operação.

## Importando as Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn import preprocessing
from joblib import dump, load
from sklearn import tree
import warnings
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import lightgbm as lgb

In [ ]:
warnings.filterwarnings('ignore') # Remoção apenas para fins estéticos do Notebook

## Importação dos Dados

* Antes de tudo vamos importar o csv "liver_cirrhosis_v3". Essa versão foi separada previamente durante a etapa de limpeza de dados, removendo algumas variáveis da tabela e deixando apenas as mais relevantes para a classificação de casos de morte. Esses atributos são:

    - `Situação (Sobreviveu, Transplante ou Morte)`
    - `Idade (dias)`
    - `Sexo (M ou F)`
    - `Bilirrubina (mg/dl)`
    - `Albumina (gm/dl)`
    - `Fosfatase Alcalina (U/L)`
    - `Aspartato Aminotransferase (U/L)`
    - `Plaquetas (ml/1000)`
    - `Tempo de Protrombina (s)`
    - `Estágio (1, 2 ou 3)`

* Para fins de legibilidade e funcionamento dos códigos do notebook, vamos modificar a coluna Sexo com um label encoder do sklearn, para que ao invés de ser uma coluna de strings (M ou F), ela seja uma coluna binária (0 ou 1). Nesse caso, as linhas com valor 0 serão do sexo feminino e com valor 1 serão do sexo masculino (essa é a ordem que o label encoder coloca por padrão). Por isso, ela também será renomeada para Sexo Masculino, já que: 0 (false) = Feminino e 1 (true) = Masculino.

In [ ]:
df = pd.read_csv("../data/dados_processados/liver_cirrhosis_v3.csv")

# Instanciar o Label Encoder para converter as colunas String
label_encoder = preprocessing.LabelEncoder()
df['Sexo'] = label_encoder.fit_transform(df['Sexo'])

df.rename(
    columns={'Sexo':'Sexo Masculino'}, # Mudando 'Sexo' para 'Sexo Masculino', 
    inplace=True # Realiza a mudança no prórprio df
)

df

---
# Classificação 1: Morte, Sobrevivência e Transplante
---

## **Importação e Ajuste dos Dados**

* Com o dataset extraído, a próxima etapa será dividir as variáveis *X* e *y*. Como queremos classificar a coluna *Situação* e prever os casos de *sobrevivência*, *morte* e *transplante*", vamos definir essa coluna como sendo a *y*. O resto das colunas usaremos como valores de entrada sendo a variável *X*.

In [ ]:
df1 = df.copy()

# Obtendo as variáveis x e y para treinar o modelo 
X1  = df1.drop(columns=['Situação']).values
y1 = df1['Situação'].values

print(X1)
print(y1)

## **Separação e Normalização dos Dados**

* Vamos dividir cada uma de nossas variáveis em duas partes, uma fração para treino e outra para testes. Nesse caso, decidimos dividir 80% dos valores para treino e o restante, que seria 20%, para os testes mais para frente.

* Seguindo, agora é hora de normalizar os dados que separamos. Para isso, é utilizado o objeto **StandardScaler** e seus métodos *fit_transform()* e *fit()*. Depois de aplicada a normalização, nós salvamos o scaler na pasta *scalers* com o nome *death_scaler.joblib* para possíveis usos futuros.

In [ ]:
# Dividindo 80% da matriz para treinar o modelo e 20% para os testes
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2)
print("Tamanho separado para treinos: " + str(len(X_train1)))
print("Tamanho separado para testes: " + str(len(X_test1)) + "\n")

# Normalizando os dados extraídos do dataset
scaler1 = StandardScaler()
X_train1_scaled = scaler1.fit_transform(X_train1)
X_test1_scaled = scaler1.transform(X_test1)

# Salvando o scaler para uso futuro
print("O modelo do scaler foi salvo em:")
dump(scaler1, 'scalers/scalers_morte/death_scaler1.joblib')

## **Regressão Logística**

* Nosso primeiro modelo de classificação será utilizando Regressão Logística. Vamos utilizar o objeto **LogisticRegression** e dar um *fit()* para poder treinar nosso modelo, e depois salvar a predição numa variável separada. Os resultados da regressão são exibidos através de uma tabela de resultados, com a precisão de cada classe e a acurácia geral, e também uma matriz de confusão com as três classes.

* Por fim, o modelo de Regressão Logística é salvo na pasta *models* no arquivo nomeado *death_logreg1.joblib* para que seja mais fácil acessá-lo caso seja necessário.

In [ ]:
# Instanciando o modelo
logreg1 = LogisticRegression(max_iter=4000, n_jobs=-1, C=0.1, penalty='l2')
logreg1.fit(X_train1_scaled, y_train1)

# Predict com dados de treino
y_pred1T = logreg1.predict(X_train1_scaled)

# Tabela de Resultados
print("Report - Regressão Logística - Treino:")
print(classification_report(y_train1, y_pred1T, zero_division=0))

In [ ]:
# Predict do teste
y_pred1 = logreg1.predict(X_test1_scaled)

# Tabela de Resultados
print("Report - Regressão Logística - Teste:")
print(classification_report(y_test1, y_pred1, zero_division=0))

* Na regressão logística, estamos obtendo uma acurácia na faixa dos 70%, o que não é tão ruim, mas poderia ser bem melhor. Isso se dá, em partes devido à natureza sintética dos dados encontrados no dataset. Outro fator que também torna esse modelo inadequado é a exclusão total da classe **Transplante**: perceba que ela não está sendo classificada. Isso se deve ao desbalanceamento do dataset: tem muitos poucos casos com essa classe comparada às outras, algo que torna o desempenho do algoritmo de regressão logística impreciso.

In [ ]:
# Importância dos coeficientes
coefficients1 = logreg1.coef_
importance1 = pd.DataFrame(coefficients1.T, columns = logreg1.classes_, index=df1.drop('Situação', axis=1).columns)
print("\nImportância dos coeficientes:")
print(importance1)

* Vamos avaliar os 3 atributos mais importantes para cada classe.

    * **Morte**
        - Idade: O aumento na idade dos pacientes tende a torná-los mais vulneráveis, portanto é impactante pra classificar os casos de morte.
        - Bilirrubina (mg/dl): Fortemente associada ao mal funcionamento do fígado, logo também é bastante impactante.
        - Fosfatase Alcalina (U/L): Importância diretamente proporcional, pois taxas altas desse químico sugerem obstruções no fluxo do fígado.

    * **Sobreviveu**
        - Bilirrubina (mg/dl): Oposta a importância que teve na classe Morte, já que as taxas menores significam que o fígado está mais saudável.
        - Aspartato Aminotransferase (U/L): Valores altos indicam dano nas células do fígado, portanto valores menores importam para os casos de sobrevivência.
        - Estágio: Quanto menor o estágio, mais fácil é do paciente sobreviver a doença. Portanto é de se esperar que valores baixos tenham mais importância aqui.
        
    * **Transplante**
        - Idade: Perceba que tem importância pro lado negativo, sugerindo que as pessoas tendem a optar pelo transplante em idades mais jovens.
        - Bilirrubina (mg/dl): Influência positiva, assim como na classe Morte. Pode sugerir que pessoas com altas taxas procuram o transplante como solução.
        - Fosfatase Alcalina (U/L): Diferente das outras duas classes, aqui tem importância pro lado negativo, refletindo a possível melhora no funcionamento do fígado após o transplante.

* Criar matrizes de confusão nos ajuda a visualizar melhor as distribuições das classificações dos nossos modelos, então vamos fazer isso para cada um dos algoritmos no notebook.

In [ ]:
# Matriz de Confusão
matrix1 = confusion_matrix(y_test1, y_pred1)

plt.figure(figsize=(6, 6))

matrixPlot1 = ConfusionMatrixDisplay(confusion_matrix=matrix1, display_labels=logreg1.classes_)
matrixPlot1.plot(cmap=plt.cm.Blues)

plt.title('Regressão Logística - Matriz de Confusão')
plt.xlabel('Prevista')
plt.ylabel('Real')

plt.show()

* Perceba que na 1° linha, a de Morte, a matriz apresenta valores dispersos entre a 1° e 2° coluna. Isso se dá a baixa taxa de recall que o modelo desenvolveu, refletindo que nosso modelo está prevendo casos que deveriam ser de Morte como Sobreviveu, erroneamente.

* Em Transplante é possivel verificar que o modelo não foi capaz de classificar corretamente. Isso pode ter ocorrido pela observação que citamos anteriormente, não existem dados suficientes de transplante para que seja possivel a classificação.

In [ ]:
# Salvando o modelo de regressão para uso futuro
print("O modelo de Regressão Logística foi salvo em:")
dump(logreg1, 'models/modelos_morte/death_logreg1.joblib')

## **Árvore de Decisão**

* Agora vamos realizar outra alternativa de classificação. Dessa vez, vamos utilizar o algoritmo de Árvore de Decisão, que pode ser útil para nos ajudar a visualizar melhor o que está acontecendo no processo de classificação, e talvez até entregue resultados melhores na acurácia e precisão.

* Perceba que podemos utilizar as mesmas variáveis X e y, já que não foram modificadas.

In [ ]:
# Instanciar a Árvore de Decisão
clf1 = DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_leaf=5)

clf1.fit(X_train1, y_train1)

# Predict com dados de treino
tree_pred1T = clf1.predict(X_train1)

# Tabela de Resultados
print("Report - Árvore de Decisão - Treino:")
print(classification_report(y_train1, tree_pred1T, zero_division=0))

In [ ]:
# Predict da Árvore de Decisão
tree_pred1 = clf1.predict(X_test1)

# Tabela de Resultados
print("Report - Árvore de Decisão - Teste:")
print(classification_report(y_test1, tree_pred1, zero_division=0))

* Perceba que a acurácia aumentou consideravelmente, de aproximadamente 70% para aproximadamente 80%. Além disso, a classe **Transplante**, que antes estava sendo omitida da classificação, agora está sendo classificada corretamente. Maravilha! Além disso, aqui não está tendo overfitting já que a acurácia se mantém similar entre a classificação com dados de treino e teste.

In [ ]:
# Importância dos coeficientes
importances1 = clf1.feature_importances_
feature_importances1 = pd.DataFrame(
    importances1,
    columns = ['Importance'],
    index=df1.drop('Situação', axis=1).columns
).sort_values(by='Importance', ascending=False)

print("\nImportância das Features:")
print(feature_importances1)

* Forte Importância:
    - Bilirrubina(mg/dl): 0.627105
* Importância Moderada:
    - Aspartato_Aminotransferase(U/L): 0.111300
    - Fosfatase_Alcalina (U/L): 0.104742
    - Albumina(gm/dl): 0.085476
* Importância Fraca:
    - Tempo_de_Protrombina(s): 0.040876
    - Estágio: 0.030501
    - Idade: 0.000000
    - Plaquetas(ml/1000): 0.000000

* Agora vamos plotar a nossa árvore de decisão, para que possamos visualizar as etapas que o modelo tomou para alcançar tais resultados.

In [ ]:
# Plottar a Árvore de Decisão
fig1 = plt.figure(figsize=(90,40))
_ = tree.plot_tree(clf1,
                      feature_names=df1.drop(columns=['Situação']).columns,
                      class_names=clf1.classes_,
                      filled=True,
                      fontsize=10,
                      )

plt.show()

In [ ]:
# Matriz de Confusão
matrixTree1 = confusion_matrix(y_test1, tree_pred1)

plt.figure(figsize=(6, 6))

matrixTreePlot1 = ConfusionMatrixDisplay(confusion_matrix=matrixTree1, display_labels=clf1.classes_)
matrixTreePlot1.plot(cmap=plt.cm.Blues)

plt.title('Árvore de Decisão - Matriz de Confusão')
plt.xlabel('Prevista')
plt.ylabel('Real')

plt.show()

* A matriz de confusão da árvore, diferentemente da matriz de regressão, agora possui valores na coluna *Transplantes*, o que quer dizer que nosso modelo está finalmente sendo capaz de prever e distinguir casos onde os pacientes realizaram um transplante. Ademais, a matriz está mais definida, com valores concentrados na diagonal principal, o que quer dizer que a nossa acurácia está bem alta, prevendo corretamente a grande maioria dos casos. 

### Verificando os resultados da Árvore de Decisão

In [ ]:
y_test1[645]

In [ ]:

def extract_rules(clf, x_sample):
    #Id das features analisadas em cada nó da árvore de decisão
    feature = clf.tree_.feature

    #Limiar de decisão de cada nó da árvore
    threshold = clf.tree_.threshold

    #Acessa o caminho de nós da árvore até a folha de predicao da amostra
    node_indices = clf.decision_path([x_sample]).indices 
   
    #Último nó do caminho é a folha de predição
    leaf_id = node_indices[-1]
   
    print('\nFeatures usadas para predizer a amostra')

    for f, v in zip(df.drop(columns=['Situação']).columns, x_sample):
        print('%s = %s'%(f,v))
    print('\n')      

    for node_id in node_indices:
        if leaf_id == node_id:
            break


        if (x_sample[feature[node_id]] <= threshold[node_id]):
            threshold_sign = "<="
        else:
            threshold_sign = ">"

        print("id do nó de decisão %s : (atributo %s com valor = %s %s %s)"
              % (node_id,
                 df.drop(columns=['Situação']).columns[feature[node_id]],
                 x_sample[feature[node_id]],
                 threshold_sign,
                 threshold[node_id]))
        
    pred = clf.predict([x_sample])

    print("\tClasse => %s" %pred)


extract_rules(clf1, X_test1[645])

## **Reestruturação da Árvore de Decisão**

* A nossa árvore de decisão está entregando bons resultados, mas está bem ilegível. Para corrigir isso, vamos modificar alguns parâmetros para que a análise fique mais sucinta (e de fato possível).

* Na primeira árvore, nós utilizamos valores selecionados a dedo por um processo de tentativa e erro para tentar torná-la aceitável. Como solução mais prática, podemos realizar uma *Grid Search* para automaticamente encontrar o melhor valor para os parâmetros da árvore de decisão. No código, ao lado de cada parâmetro há um comentário explicando brevemente a função de cada parâmetro.

In [ ]:
# Grid Search
path1 = clf1.cost_complexity_pruning_path(X_train1, y_train1)

param_grid1 = {
    'max_depth': range(2, 10), # Profundidade da árvore, limita o quanto a árvore pode crescer
    'min_samples_split': range(2, 10), # A quantidade mínima de samples para separar cada caminho da árvore
    'min_samples_leaf': range(1, 5), # O mínimo de samples que deve estar em cada folha da árvore
    'criterion': ['gini', 'entropy'], # Critérios de partição da árvore
    'max_features': range(4, X1.shape[1] + 1), # Limita o número de características avaliadas
    'ccp_alpha': (0.0, 0.1, 0.5, 0.05, 0.005) # Penalizador utilizado para regular a árvore
}

CV_clf = GridSearchCV(estimator=clf1, param_grid=param_grid1, cv = 7, verbose=2, n_jobs=-1)
CV_clf.fit(X_train1, y_train1)

best_max_depth1 = CV_clf.best_estimator_.max_depth
best_min_split1 = CV_clf.best_estimator_.min_samples_split
best_min_leaf1 = CV_clf.best_estimator_.min_samples_leaf
best_criterion1 = CV_clf.best_estimator_.criterion
best_max_features1 = CV_clf.best_estimator_.max_features
best_ccp_alpha1 = CV_clf.best_estimator_.ccp_alpha

* Com os valores em mãos, agora vamos criar outra árvore de decisão com os parâmetros modificados, com o objetivo de torná-la mais agradável esteticamente sem que haja uma diminuição muito grande da acurácia.

In [ ]:
# Instanciar um novo objeto para a Árvore de Decisão Reestruturada
clf_ccp1 = DecisionTreeClassifier(criterion=best_criterion1,
                                   max_depth=6, # O resultado do GridSearch acaba tornando a árvore muito ilegível, então vamos manter 6 aqui
                                   min_samples_leaf=best_min_leaf1, 
                                   min_samples_split=best_min_split1, 
                                   ccp_alpha=best_ccp_alpha1,
                                   max_features=best_max_features1)

clf_ccp1.fit(X_train1, y_train1)

# Predict da Árvore de Decisão Reestruturada - Treino
tree_pred1T = clf_ccp1.predict(X_train1)

# Tabela de Resultados
print("Report - Árvore de Decisão Reestruturada - Dados de Treino:")
print(classification_report(y_train1, tree_pred1T, zero_division=0))

In [ ]:
# Predict da Árvore de Decisão Reestruturada - Teste
tree_pred1 = clf_ccp1.predict(X_test1)

# Tabela de Resultados
print("Report - Árvore de Decisão Reestruturada - Teste:")
print(classification_report(y_test1, tree_pred1, zero_division=0))

* Com as limitações que colocamos na árvore, a acurácia agora parece se estabilizar na faixa dos 80%, o que é ótimo pois não houve uma diminuição tão considerável da árvore anterior e ainda assim conseguimos tornar a árvore bem mais interpretável. O modelo se mantém sem overfitting.

In [ ]:
# Importância dos coeficientes
importances1 = clf_ccp1.feature_importances_
feature_importances1 = pd.DataFrame(
    importances1,
    columns = ['Importance'],
    index=df1.drop('Situação', axis=1).columns
).sort_values(by='Importance', ascending=False)

print("\nImportância das Features:")
print(feature_importances1)

In [ ]:
# Árvore de Decisão Reestruturada
fig_1 = plt.figure(figsize=(90,40))
_ = tree.plot_tree(clf_ccp1,
                      feature_names=df1.drop(columns=['Situação']).columns,
                      class_names=clf_ccp1.classes_,
                      filled=True,
                      fontsize=10,
                      )

plt.show()

In [ ]:
# Matriz de Confusão
matrixTree1 = confusion_matrix(y_test1, tree_pred1)

plt.figure(figsize=(6, 6))

matrixTreePlot1 = ConfusionMatrixDisplay(confusion_matrix=matrixTree1, display_labels=clf_ccp1.classes_)
matrixTreePlot1.plot(cmap=plt.cm.Blues)

plt.title('Árvore de Decisão Reestruturada - Matriz de Confusão')
plt.xlabel('Prevista')
plt.ylabel('Real')

plt.show()

* Nessa matriz de confusão, podemos notar que, infelizmente, o recall da classe *Transplante* diminuiu, mas isso é consequência do processo que fizemos para tornar a árvore mais legível. Mesmo assim, o elemento 3x3 ainda possui a maior quantia da linha 3, o que significa que, em maioria, o modelo está prevendo corretamente os casos de transplante. 

Para finalizar, vamos salvar arquivos dos nossos modelos de árvore de decisão para que possamos usá-las em outra situação.

In [ ]:
print("O modelo de Árvore de Decisão Corrigida foi salvo em:")
dump(clf_ccp1, 'models/modelos_morte/death_tree_fixed1.joblib')

## **Rede Neural**

* Nessa seção, vamos mudar o foco para utilizar um modelo de Rede Neural nas previsões. Como a rede neural exige que as classes sejam números, e não strings, é necessário fazer um **label encoding** com a biblioteca do sklearn para deixar a coluna *Situação* em números.

In [ ]:
df1N = df.copy()
df1N['Situação'] = label_encoder.fit_transform(df1N['Situação'])

X1N  = df1N.drop(columns=['Situação']).values
y1N = df1N['Situação'].values

X_train1N, X_test1N, y_train1N, y_test1N = train_test_split(X1N, y1N, test_size=0.2)
X_train1N = scaler1.fit_transform(X_train1N)
X_test1N = scaler1.transform(X_test1N)

* Aqui estamos inicializando uma rede neural com:

    * camada de entrada com 32 neurônios e função de ativação *relu*;

    * duas camadas com 64 neurônios;

    * uma camada com 32 neurônios;

    * camada de saída com 2 neurônios (para as duas classes).

In [ ]:
neural1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train1N.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

In [ ]:
neural1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
neural1.summary()

* A partir dessa célula, temos o treinamento do modelo de fato. Vamos rodar 200 épocas de 64 amostras cada, para conseguir fazer um treino relativamente rápido e eficiente.

In [ ]:
history = neural1.fit(X_train1N, y_train1N, epochs=200, batch_size=64, validation_data=(X_test1N, y_test1N))

* Perceba que a acurácia parece parar na faixa dos 85%. Poderíamos continuar, mas não iria mudar tanto. Abaixo, gráficos que facilitam visualizar o crescimento da acurácia na nossa rede neural.

In [ ]:
# Extract training and validation accuracy/loss
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot accuracy
plt.figure(figsize=(8, 5))
plt.plot(train_acc, label='Acurácia do Treino')
plt.plot(val_acc, label='Acurácia dos Testes')
plt.title('Curva das Acurácias (Melhor Modelo)')
plt.xlabel('Epoch')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)

In [ ]:
# Plot loss (error)
plt.figure(figsize=(8, 5))
plt.plot(train_loss, label='Perda do Treino')
plt.plot(val_loss, label='Perda dos Testes')
plt.title('Curvas das Perdas (Melhor Modelo)')
plt.xlabel('Epoch')
plt.ylabel('Perda')
plt.legend()
plt.grid(True)

* Depois de tudo isso, printar os reports da classificação para analisar os resultados finais do nosso modelo (e novamente checar se está acontecendo overfitting)

In [ ]:
print("Report - Rede Neural - Treino:")
print(classification_report(y_train1N, np.argmax(neural1.predict(X_train1N), axis=1), zero_division=0))

In [ ]:
print("Report - Rede Neural - Teste:")
print(classification_report(y_test1N, np.argmax(neural1.predict(X_test1N), axis=1), zero_division=0))

* Enfim, vamos salvar o modelo de Rede Neural

In [ ]:
print("O modelo de Rede Neural foi salvo em:")
dump(neural1, 'models/modelos_morte/death_neural1.joblib')

## **Light Gradient Boosting Machine (LGBM)**

* Como uma análise final, vamos finalmente realizar a classificação utilizando dessa vez o **Light Gradient Boosting Machine** (ou LGBM). Esse modelo, diferente dos outros, faz um processo de ensemble de tipo boosting, que vai combinar vários modelos do mesmo algoritmo de forma sequencial para tornar o modelo muito mais eficaz e prático.

In [ ]:
# Utilizando as mesmas variáveis da Rede Neural, pois já estão reguladas pelo label encoder
train_data1 = lgb.Dataset(X_train1N, label=y_train1N)
valid_data1 = lgb.Dataset(X_test1N, label= y_test1N)

# Define os parâmetros do modelo
parameters1 = {
    'objective': 'multiclass',  # Define o objetivo como classificação multiclasse
    'boosting_type': 'gbdt',  # Usa Gradient Boosting Decision Tree
    'metric': 'multi_logloss',  # Métrica de avaliação: log-loss para múltiplas classes
    'num_class': len(np.unique(y_test1N)),  # Número de classes únicas em 'Situação'
    'num_leaves': 31,  # Número máximo de folhas em cada árvore
    'learning_rate': 0.05,  # Taxa de aprendizado do modelo
    'feature_fraction': 0.9  # Fração de features usadas em cada iteração de construção da árvore
}

# Treina o modelo
model_lgbm1 = lgb.train(parameters1, train_data1, valid_sets=[train_data1, valid_data1], num_boost_round=5000)

In [ ]:
# Faz previsões
y_train_pred_1 = model_lgbm1.predict(X_train1N)
y_valid_pred_1 = model_lgbm1.predict(X_test1N)

# Converte as previsões para rótulos de classe
y_train_pred_1_labels = np.argmax(y_train_pred_1, axis=1)
y_valid_pred_1_labels = np.argmax(y_valid_pred_1, axis=1)

* Depois de treinar o modelo, vamos checar as tabelas para tomar conclusões.

In [ ]:
# Tabela de resultado com os dados de treino
print("Report - LGBM - Treino:")
print(classification_report(y_train1N, y_train_pred_1_labels, zero_division=0))


In [ ]:
# Resultados com os dados de
print("Report - LGBM - Teste:")
print(classification_report(y_test1N, y_valid_pred_1_labels, zero_division=0))

* Algo interessante que acontece aqui é que o modelo conseguiu alcançar uma porcentagem de **100% de acurácia** nos dados de treino! Isso normalmente poderia ser preocupante, mas perceba que a acurácia nos dados de teste se mantém na faixa dos 97%, o que é extremamente bom e similar a do treino, logo não há overfitting.

In [ ]:
# Matriz de Confusão
matrixLGBM1T = confusion_matrix(y_test1N, y_valid_pred_1_labels)
print(matrixLGBM1T)

plt.figure(figsize=(6, 6))
matrixLGBMPlot1 = ConfusionMatrixDisplay(confusion_matrix=matrixLGBM1T, display_labels=np.unique(y_test1))
matrixLGBMPlot1.plot(cmap=plt.cm.Blues)

plt.title('LightGBM - Matriz de Confusão')
plt.xlabel('Prevista')
plt.ylabel('Real')

plt.show()

* A matriz de confusão deixa ainda mais evidente a eficácia do LGBM. O recall do modelo está ótimo, já que a maior concentração das previsões está de fato onde deveriam estar, linha batendo com coluna (1x1, 2x2, 3x3). As cores fortes na diagonal principal significam que a grande maioria das previsões está batendo com os dados reais.

* Para finalizar nossa classificação, vamos salvar o modelo.

In [ ]:
# Salvando o modelo
print("O modelo de LightGBM foi salvo em:")
dump(model_lgbm1, 'models/modelos_morte/death_lightgbm1.joblib')

---
# Classificação 2: Mesclando Sobrevivência e Transplante
---

## **Importação e Ajuste dos Dados**

* Junção dos dados de *Sobreviveu* e *Transplante* em uma única classe.

In [ ]:
df2 = df.copy()

# Considerando o valor de "Transplante" como "Sobreviveu"
df2['Situação'] = df2['Situação'].replace('Transplante', 'Sobreviveu')

print(df2['Situação'].value_counts())
#df2

* A coluna **"Situação"** passa a ter apenas duas classes: *Sobreviveu* e *Morte*.

## **Separação e Normalização dos Dados**

* Dividimos as colunas nas variáveis X2 e y2.

* Separamos os dados de ambas as variáveis em 80% para treino e 20% para teste.

* Normalizamos os dados de X2 utilizando o *StandardScaler* e salvamos o scaler na pasta **scalers** para uso futuro.

In [ ]:
X2 = df2.iloc[:, 1:].values
y2 = df2.iloc[:, 0].values

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=0)

scaler2 = StandardScaler()
scaler2.fit(X_train2)
X_train2_scaled = scaler2.transform(X_train2)
X_test2_scaled = scaler2.transform(X_test2)

print("Tamanho treinamento: " + str(len(X_train2)))
print("Tamanho teste: " + str(len(X_test2)))

In [ ]:
print("O modelo do scaler foi salvo em:")
dump(scaler2, 'scalers/scalers_morte/death_scaler2.joblib')

## **Regressão Logistica**

* Nosso primeiro modelo de classificação será utilizando Regressão Logística. Vamos utilizar o objeto LogisticRegression e dar um fit() para treinar nosso modelo, e depois salvar a predição eu uma variável separada. Os resultados da regressão são exibidos através de uma tabela de resultados, com a precisão de cada classe, a acurácia geral e também uma matriz de confusão com as duas classes.

* Por fim, o modelo de Regressão Logística é salvo na pasta *models*, no arquivo nomeado *death_logreg2.joblib*, para que seja mais fácil de acessá-lo caso seja necessário.

In [ ]:
logreg2 = LogisticRegression(max_iter=4000, n_jobs=-1, C=0.1, penalty='l2')
logreg2.fit(X_train2_scaled, y_train2)

In [ ]:
# Predict do treino
y_pred2T = logreg2.predict(X_train2_scaled)

# Tabela de Resultados
print("Report - Regressão Logística - Treino:")
print(classification_report(y_train2, y_pred2T, zero_division=0))

In [ ]:
# Predict do teste
y_pred2 = logreg2.predict(X_test2_scaled)

# Tabela de Resultados
print("Report - Regressão Logística - Teste:")
print(classification_report(y_test2, y_pred2, zero_division=0))

* Pelos resultados das acurácias do treino e do teste serem similares, podemos concluir que o modelo não está sofrendo de overfitting. Apesar disso, a acurácia do modelo não é tão alta, ficando com valores por volta de 75%. Logo, podemos continuar explorando outros modelos de classificação para tentar melhorar esse valor.

In [ ]:
# Importância dos coeficientes
feature_names2 = df2.columns[1:]
coefficients2 = logreg2.coef_
coef_df2 = pd.DataFrame(coefficients2, columns=feature_names2)
for i, class_name in enumerate(logreg2.classes_):
    sorted_coef2 = coef_df2.iloc[i].sort_values(ascending=False)
    print(f"Importância das Features em relação a {class_name}:\n")
    print(sorted_coef2)
    break

* Percebemos que a *Bilirrubina*, o *Estágio* e o *Tempo de Protrombina* são as variáveis mais influentes para a classificação. Isso ocorre pois a bilirrubina é um indicador que está relacionado à falha da função hepática ou obstrução biliar, o estágio da cirrose é um indicador da gravidade da doença e o tempo de protrombina é um indicador da capacidade de coagulação do sangue. Portanto, quanto maior a magnitude dessas variáveis, maior a probabilidade de morte.

In [ ]:
# Matriz de Confusão
matrix2 = confusion_matrix(y_test2, y_pred2)

plt.figure(figsize=(6, 6))

matrixPlot2 = ConfusionMatrixDisplay(confusion_matrix=matrix2, display_labels=logreg2.classes_)
matrixPlot2.plot(cmap=plt.cm.Blues)

plt.title('Regressão Logística - Matriz de Confusão')
plt.xlabel('Prevista')
plt.ylabel('Real')

plt.show()

* A matriz nos mostra que, apesar de acertar a maioria dos casos de sobrevivência, o modelo tem dificuldade em classificar os casos de morte.

In [ ]:
# Salvando o modelo de regressão para uso futuro
print("O modelo de Regressão Logística foi salvo em:")
dump(logreg2, 'models/modelos_morte/death_logreg2.joblib')

## **Árvore de Decisão**

* Agora vamos tentar outra alternativa de classificação. Dessa vez, vamos tentar utilizar o algoritmo de Árvore de Decisão, que pode ser útil para nos ajudar a visualizar melhor o que está acontecendo no processo de classificação e, possivelmente entregue resultados melhores na acurácia e precisão.

* Perceba que podemos utilizar as mesmas variáveis X e y, já que não foram modificadas.

In [ ]:
X_train_tree2, X_test_tree2, y_train_tree2, y_test_tree2 = train_test_split(X2, y2, test_size=0.2, random_state=0)

In [ ]:
# Instanciar a Árvore de Decisão
clf2 = DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_leaf=5)

clf2.fit(X_train_tree2, y_train_tree2)

In [ ]:
# Predict da Árvore de Decisão - Treino
tree_pred2T = clf2.predict(X_train_tree2)

# Tabela de Resultados
print("Report - Árvore de Decisão - Treino:")
print(classification_report(y_train_tree2, tree_pred2T, zero_division=0))

In [ ]:
# Predict da Árvore de Decisão - Teste
tree_pred2 = clf2.predict(X_test_tree2)

# Tabela de Resultados
print("Report - Árvore de Decisão - Teste:")
print(classification_report(y_test_tree2, tree_pred2, zero_division=0))

* Pelos resultados das acurácias do treino e do teste serem similares, podemos concluir que o modelo não está sofrendo de overfitting. Além disso, percebemos que a acurácia do modelo melhorou, ficando na faixa dos 85%. Porém, apesar de apresentar melhora significativa em relação ao modelo de Regressão Logística, a acurácia pode ser melhorada.

In [ ]:
# Importância dos coeficientes
importances2 = clf2.feature_importances_
feature_importances2 = pd.DataFrame(
    importances2,
    columns = ['Importance'],
    index=df2.drop('Situação', axis=1).columns
).sort_values(by='Importance', ascending=False)

print("\nImportância das Features:")
print(feature_importances2)

* Neste modelo, notamos que a *Bilirrubina* é a variável mais influente para a classificação. Além disso, podemos perceber que as demais variáveis possuem um impacto menor na classificação quando comparados ao modelo de Regressão Logística.

In [ ]:
# Plottar a Árvore de Decisão
fig2 = plt.figure(figsize=(90,40))
_ = tree.plot_tree(clf2,
                      feature_names=df2.drop(columns=['Situação']).columns,
                      class_names=clf2.classes_,
                      filled=True,
                      fontsize=10,
                      )

plt.show()

In [ ]:
# Matriz de Confusão
matrixTree2 = confusion_matrix(y_test_tree2, tree_pred2)

plt.figure(figsize=(6, 6))

matrixTreePlot2 = ConfusionMatrixDisplay(confusion_matrix=matrixTree2, display_labels=clf2.classes_)
matrixTreePlot2.plot(cmap=plt.cm.Blues)

plt.title('Árvore de Decisão - Matriz de Confusão')
plt.xlabel('Prevista')
plt.ylabel('Real')

plt.show()

* A matriz de confusão da árvore, diferentemente da matriz de regressão, agora possui muito mais valores de morte corretamente classificados. Além disso, a matriz está bem mais definida, com valores concentrados na diagonal principal, o que quer dizer que a nossa acurácia está melhor.

### Verificando os resultados da Árvore de Decisão

In [ ]:
y_test1[123]

In [ ]:
extract_rules(clf2, X_test2[123])

## **Reestruturação da Árvore de Decisão**

* A nossa Árvore de Decisão está entregando bons resultados, mas ainda pode melhorar. Para isso, vamos usar o GridSearchCV para encontrar os melhores parâmetros.

In [ ]:
# Grid Search
path2 = clf2.cost_complexity_pruning_path(X_train_tree2, y_train_tree2)

param_grid2 = {
    'max_depth': range(2, 10),
    'min_samples_split': range(2, 10),
    'min_samples_leaf': range(1, 5),
    'criterion': ['gini', 'entropy'],
    'max_features': range(4, X2.shape[1] + 1),
    'ccp_alpha': (0.0, 0.1, 0.5, 0.05, 0.005)
}

CV_clf = GridSearchCV(estimator=clf2, param_grid=param_grid2, cv = 7, verbose=2, n_jobs=-1)
CV_clf.fit(X_train_tree2, y_train_tree2)

best_max_depth2 = CV_clf.best_estimator_.max_depth
best_min_split2 = CV_clf.best_estimator_.min_samples_split
best_min_leaf2 = CV_clf.best_estimator_.min_samples_leaf
best_criterion2 = CV_clf.best_estimator_.criterion
best_max_features2 = CV_clf.best_estimator_.max_features
best_ccp_alpha2 = CV_clf.best_estimator_.ccp_alpha

* Com os melhores valores, agora podemos criar outra árvore de decisão com tais parâmetros modificados, com o objetivo de melhorar a apresentação da Árvore sem afetar, de forma significativa a acurácia.

In [ ]:
# Instanciar um novo objeto para a Árvore de Decisão Reestruturada
clf_ccp2 = DecisionTreeClassifier(criterion=best_criterion2,
                                   max_depth=6, # O resultado do GridSearch acaba tornando a árvore muito ilegível, então vamos manter 6 aqui
                                   min_samples_leaf=best_min_leaf2, 
                                   min_samples_split=best_min_split2, 
                                   ccp_alpha=best_ccp_alpha2,
                                   max_features=best_max_features2)

clf_ccp2.fit(X_train_tree2, y_train_tree2)

* Apesar do *best_max_depth2* ser 9, a visualização da árvore continua complicada. Por isso, vamos limitar *max_depth* em 6 para que a árvore seja mais legível.

In [ ]:
# Predict da Árvore de Decisão Reestruturada - Treino
tree_pred_ccp2T = clf_ccp2.predict(X_train_tree2)

# Tabela de Resultados
print("Report - Árvore de Decisão Reestruturada - Treino:")
print(classification_report(y_train_tree2, tree_pred_ccp2T, zero_division=0))

In [ ]:
# Predict da Árvore de Decisão Reestruturada - Teste
tree_pred_ccp2 = clf_ccp2.predict(X_test_tree2)

# Tabela de Resultados
print("Report - Árvore de Decisão Reestruturada - Teste:")
print(classification_report(y_test_tree2, tree_pred_ccp2, zero_division=0))

* Com os parâmetros que colocamos na árvore, a acurácia permanece parecida com a da árvore anterior e ainda se mantém sem overfitting.

In [ ]:
# Importância dos coeficientes
importances_ccp2 = clf_ccp2.feature_importances_
feature_importances_ccp2 = pd.DataFrame(
    importances_ccp2, columns = ['Importance'], index=df2.drop('Situação', axis=1).columns).sort_values(by='Importance', ascending=False)

print("\nImportância das Features:")
print(feature_importances_ccp2)

* Assim como no modelo anterior, notamos que a *Bilirrubina* é a variável mais influente para a classificação.

In [ ]:
# Árvore de Decisão Reestruturada
fig_2 = plt.figure(figsize=(90,40))
_ = tree.plot_tree(clf_ccp2,
                      feature_names=df2.drop(columns=['Situação']).columns,
                      class_names=clf_ccp2.classes_,
                      filled=True,
                      fontsize=10,
                      )

plt.show()

In [ ]:
# Matriz de Confusão
matrixTree2T = confusion_matrix(y_test_tree2, tree_pred2)

plt.figure(figsize=(6, 6))

matrixTreePlot2 = ConfusionMatrixDisplay(confusion_matrix=matrixTree2T, display_labels=clf_ccp2.classes_)
matrixTreePlot2.plot(cmap=plt.cm.Blues)

plt.title('Árvore de Decisão Reestruturada - Matriz de Confusão')
plt.xlabel('Prevista')
plt.ylabel('Real')

plt.show()

* A Matriz de Confusão da Árvore Reestruturada permanece semelhante à matriz anterior.

In [ ]:
# Salvando o modelo de árvore de decisão para uso futuro

print("O modelo de Árvore de Decisão Reestruturada foi salvo em:")
dump(clf_ccp2, 'models/modelos_morte/death_tree_fixed2.joblib')

## **Rede Neural**

* Nessa seção, vamos mudar o foco para utilizar um modelo de Rede Neural nas previsões. Como a rede neural exige que as classes sejam números e não strings, é necessário fazer um *label encoding* com a biblioteca do sklearn para converter a coluna **Situação** em números.

In [ ]:
df2N = df2.copy()
df2N['Situação'] = label_encoder.fit_transform(df2N['Situação'])

X2N  = df2N.drop(columns=['Situação']).values
y2N = df2N['Situação'].values

X_train2N, X_test2N, y_train2N, y_test2N = train_test_split(X2N, y2N, test_size=0.2)
X_train2N = scaler2.fit_transform(X_train2N)
X_test2N = scaler2.transform(X_test2N)

In [ ]:
neural2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train2N.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='softmax')
])

* Aqui estamos inicializando uma rede neural com:

    * camada de entrada com 32 neurônios e função de ativação *relu*;

    * duas camadas com 64 neurônios;

    * uma camada com 32 neurônios;

    * camada de saída com 2 neurônios (para as duas classes).

In [ ]:
neural2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
neural2.summary()

* A partir dessa célula, temos o treinamento do modelo de fato. Vamos rodar 200 épocas de 64 amostras cada, para conseguir fazer um treino relativamente rápido e eficiente.

In [ ]:
history2 = neural2.fit(X_train2N, y_train2N, epochs=200, batch_size=64, validation_data=(X_test2N, y_test2N))

In [ ]:
# Extract training and validation accuracy/loss
train_acc2 = history2.history['accuracy']
val_acc2 = history2.history['val_accuracy']
train_loss2 = history2.history['loss']
val_loss2 = history2.history['val_loss']

# Plot accuracy
plt.figure(figsize=(8, 5))
plt.plot(train_acc2, label='Training Accuracy')
plt.plot(val_acc2, label='Validation Accuracy')
plt.title('Accuracy Curves (Best Model)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

* Com o gráfico anterior, podemos ver que a acurácia se estabiliza em torno de 85%, portanto, não é necessário continuar com mais épocas.

In [ ]:
# Plot loss (error)
plt.figure(figsize=(8, 5))
plt.plot(train_loss2, label='Training Loss')
plt.plot(val_loss2, label='Validation Loss')
plt.title('Loss Curves (Best Model)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

In [ ]:
print("Report - Rede Neural - Treino:")
print(classification_report(y_train2N, np.argmax(neural2.predict(X_train2N), axis=1), zero_division=0))

In [ ]:
print("Report - Rede Neural - Teste:")
print(classification_report(y_test2N, np.argmax(neural2.predict(X_test2N), axis=1), zero_division=0))

* Após a análise das acurácias do treino e do teste, percebemos que o modelo também não está sofrendo de overfitting. Além disso, a acurácia do modelo aumentou consideravelmente, o que é um bom resultado, apesar de ainda haver possibilidade de melhora.

In [ ]:
print("O modelo de Rede Neural foi salvo em:")
dump(neural2, 'models/modelos_morte/death_neural2.joblib')

## **Light Gradient Boosting Machine (LGBM)**

* Como uma análise final, vamos realizar a classificação utilizando o Light Gradient Boosting Machine (LGBM). Esse modelo, diferente dos outros, faz um processo de ensemble, que utiliza árvore de decisaã para tornar o modelo mais eficaz e prático.

In [ ]:
# Cria os datasets para LightGBM
train_data2 = lgb.Dataset(X_train2N, label=y_train2N)
valid_data2 = lgb.Dataset(X_test2N, label= y_test2N)

# Define os parâmetros do modelo
parameters2 = {
    'objective': 'multiclass',  # Define o objetivo como classificação multiclasse
    'boosting_type': 'gbdt',  # Usa Gradient Boosting Decision Tree
    'metric': 'multi_logloss',  # Métrica de avaliação: log-loss para múltiplas classes
    'num_class': len(np.unique(y2N)),  # Número de classes únicas em 'Situação'
    'num_leaves': 16,  # Número máximo de folhas em cada árvore
    'learning_rate': 0.05,  # Taxa de aprendizado do modelo
    'feature_fraction': 0.9  # Fração de features usadas em cada iteração de construção da árvore
}

# Treina o modelo
model_lgbm2 = lgb.train(parameters2, train_data2, valid_sets=[train_data2, valid_data2], num_boost_round=5000)


In [ ]:
# Faz previsões
y_train_pred_2 = model_lgbm2.predict(X_train2N)
y_valid_pred_2 = model_lgbm2.predict(X_test2N)

# Converte as previsões para rótulos de classe
y_train_pred_2_labels = np.argmax(y_train_pred_2, axis=1)
y_valid_pred_2_labels = np.argmax(y_valid_pred_2, axis=1)


In [ ]:
# Avalia o modelo no conjunto de treino
print("Report - LGBM - Treino:")
print(classification_report(y_train2N, y_train_pred_2_labels, zero_division=0))

In [ ]:
# Avalia o modelo
print("Report - LGBM - Teste:")
print(classification_report(y_test2N, y_valid_pred_2_labels, zero_division=0))

* Algo interessante que acontece aqui é que o modelo conseguiu alcançar uma porcentagem de 100% de acurácia nos dados de treino! Isso normalmente poderia ser preocupante, mas perceba que a acurácia, a precisão e o recall nos dados se mantém na similares no treino e no teste, logo, não há overfitting.

In [ ]:
# Matriz de Confusão
matrixLGBM2T = confusion_matrix(y_test2N, y_valid_pred_2_labels)

plt.figure(figsize=(6, 6))
matrixLGBMPlot2 = ConfusionMatrixDisplay(confusion_matrix=matrixLGBM2T, display_labels=np.unique(y2N))
matrixLGBMPlot2.plot(cmap=plt.cm.Blues)

plt.title('LightGBM - Matriz de Confusão')
plt.xlabel('Prevista')
plt.ylabel('Real')

plt.show()

* Por ser o modelo de previsão que possui a maior acurácia, a matriz de confusão do LGBM também é a que possui a maior quantidade de valores na diagonal principal, o que significa que a grande maioria das previsões está coerente com os dados reais.

In [ ]:
print("O modelo de LightGBM foi salvo em:")
dump(model_lgbm2, 'models/modelos_morte/death_lightgbm2.joblib')

## **Comparando Modelos**

* A seguir, vamos comparar os resultados dos modelos de classificação.

In [ ]:
# Regressão Logistica
print("Report - Regressão Logística - Teste:")
print(classification_report(y_test2, y_pred2, zero_division=0))

# Árvore de Decisão
print("\nReport - Árvore de Decisão Reestruturada - Teste:")
print(classification_report(y_test_tree2, tree_pred_ccp2, zero_division=0))

# Rede Neural
print("\nReport - Rede Neural - Teste:")
print(classification_report(y_test2N, np.argmax(neural2.predict(X_test2N), axis=1), zero_division=0))

# LightGBM
print("\nReport - LGBM - Teste:")
print(classification_report(y_test2N, y_valid_pred_2_labels, zero_division=0))

* Verificando a importância das variáveis

In [ ]:
# Regressão Logística
feature_names2 = df2.columns[1:]
coefficients2 = logreg2.coef_
coef_df2 = pd.DataFrame(coefficients2, columns=feature_names2)
for i, class_name in enumerate(logreg2.classes_):
    sorted_coef2 = coef_df2.iloc[i].sort_values(ascending=False)
    print(f"Importância das Features em relação a {class_name} na Regressão Logística:\n")
    print(sorted_coef2)
    break

# Árvore de Decisão
# Importância dos coeficientes
importances2 = clf2.feature_importances_
feature_importances2 = pd.DataFrame(
    importances2,
    columns = ['Importance'],
    index=df2.drop('Situação', axis=1).columns
).sort_values(by='Importance', ascending=False)

print("\nImportância das Features na Árvore de Decisão:")
print(feature_importances2)

* Após a análise dos resultados, percebemos que o modelo de LightGBM e a de Rede Neural apresentam os melhores resultados. Porém, apesar da Rede Neural apresentar um resultado satisfatório, o modelo de LGBM apresenta a maior acurácia, precisão e recall. Dessa forma, o modelo LightGBM é o mais indicado para a classificação dos dados. Álem disso, ao analisar a importância das variáveis, percebemos que a Bilirrubina é a variável que provoca o maior impacto na classificação.

---
# Classificação 3: Ignorando Evento Transplante
---

## **Importação e Ajuste dos Dados**

* Ignora dados de *Transplante* e renomea a coluna para Morte

In [ ]:
df3 = df.copy()

# Retira as linhas que possuem a coluna 'Situação' com o valor 'Transplante'
df3 = df3[df3['Situação'] != 'Transplante']

# Renomeia a coluna 'Situação' para 'Morte'
df3 = df3.rename(columns={'Situação': 'Morte'})

# Mostra as primeiras e ultimas linhas do dataset
df3

## **Separação e Normalização de dados**

* Dividimos as colunas nas variáveis X3 e Y3, com Y3 posduindo os dados da coluna *Morte* e X3 as demais.
    
* Separamos os dados de ambas as variáveis em 80% para treino e 20% para teste.

* Normalizamos os dados de X3 utilizando o *StandardScaler* e salvamos o scaler na pasta **scalers_morte** para uso futuro.

In [ ]:
# Definição das variáveis independentes e dependentes
X3 = np.array(df3.iloc[:, 1:].values)
Y3 = np.array(df3.iloc[:, 0].values)

# Divisão dos dados em treino e teste (80% treino e 20% teste)
x_train3, x_test3, y_train3, y_test3 = train_test_split(X3, Y3, test_size = 0.2)

# Normalização dos Dados                                                              
scaler3 = StandardScaler()
scaler3.fit(x_train3)
x_test3 = scaler3.transform(x_test3)
x_train3 = scaler3.transform(x_train3)

# Mostra o tamanho dos dados de treino e teste
print("Tamanho treinamento: " + str(len(x_train3)))
print("Tamanho teste: " + str(len(x_test3)))


In [ ]:
# Salva o scaler
print("O modelo do scaler foi salvo em: ")
dump(scaler3, 'scalers/scalers_morte/death_scaler3.joblib')

## **Regressão Logística**

* Nosso primeiro modelo de classificação será utilizando Regressão Logística. Vamos utilizar o objeto LogisticRegression e dar um fit() para treinar nosso modelo, e depois salvar a predição eu uma variável separada. Os resultados da regressão são exibidos através de uma tabela de resultados, com a precisão de cada classe, a acurácia geral e também uma matriz de confusão com as duas classes.

* Por fim, o modelo de Regressão Logística é salvo na pasta *modelos_morte*, no arquivo nomeado *death_logreg3.joblib*, para que seja mais fácil de acessá-lo caso seja necessário.

In [ ]:
# Instancia Regressão Logistica
logreg3 = LogisticRegression(max_iter=5000, n_jobs=-1)

# Fit do Classificador
logreg3.fit(x_train3, y_train3)

# Predict do teste
y_pred3 = logreg3.predict(x_test3)

# Predict do treino
y_pred3T = logreg3.predict(x_train3)


In [ ]:
# Tabela de Resultados de Treino
print("Report - Regressão Logística - Treino:")
print(classification_report(y_train3, y_pred3T, zero_division=0))

In [ ]:
# Tabela de Resultados
print("Report - Regressão Logística - Teste:")
print(classification_report(y_test3, y_pred3, zero_division=0))

* Pelos resultados da acurácias do treino e do teste serem similiares, podemmos descartar o overfitting, porém vemos que o valor da acurácia do teste não é satisfatório por estar em torno de 75%.

In [ ]:
# Importância dos coeficientes
feature_names3 = df3.columns[1:]
coefficients3 = logreg3.coef_
coef_df3 = pd.DataFrame(coefficients3, columns=feature_names3)
for idx, class_name in enumerate(logreg3.classes_):
    sorted_coef3 = coef_df3.iloc[idx].sort_values(ascending=False)
    print(f"Importância das Features em relação a {class_name}:\n")
    print(sorted_coef3)
    break

* No Resultado acima podemos verificar que o atributo com maior relação com a classe morte é a Bilirrubina, isso se deve ao fato deste atributo se relacionar com a falha da função hepática ou obstrução biliar.

In [ ]:
# Salva o modelo 
print("O modelo de Regressão Logística foi salvo em:")
dump(logreg3, 'models/modelos_morte/death_logreg3.joblib')


In [ ]:
# Matriz de confusão
cm = confusion_matrix(y_test3, y_pred3)

# Plota a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Morte', 'Sobreviveu'], yticklabels=['Morte', 'Sobreviveu'])
plt.xlabel('Prevista')
plt.ylabel('Real')
plt.title('Regressão Logística - Matriz de Confusão')

# Mostra a matriz de confusão
plt.show()

* Avaliando a matriz de confusão:

    * É perceptível que obtivemo um melhor desempenho para a classe Sobreviveu, isto se deve ao fato de que a classe Sobreviveu possui mais exemplos do que a classe Morte.

    * Temos quantidades próximas para a classificação correta de morte e classificação erronea de morte, isto se deve a baixa taxa de recall para a classe morte.

## **Árvore de Decisão**

In [ ]:
X_train_tree3, X_test_tree3, y_train_tree3, y_test_tree3 = train_test_split(X3, Y3, test_size=0.2, random_state=0)
# Instanciar a Árvore de Decisão
clf3 = DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_leaf=5)

clf3.fit(X_train_tree3, y_train_tree3)

# Predict da Árvore de Decisão - Teste
tree_pred3 = clf3.predict(X_test_tree3)

# Predict da Árvore de Decisão - Treino
tree_pred3T = clf3.predict(X_train_tree3)


In [ ]:
# Tabela de Resultados - Treino
print("Report - Árvore de Decisão - Treino:")
print(classification_report(y_train_tree3, tree_pred3T, zero_division=0))


In [ ]:
# Tabela de Resultados - Teste
print("Report - Árvore de Decisão - Teste:")
print(classification_report(y_test_tree3, tree_pred3, zero_division=0))

* Neste modelo a acurácia está em torno de 89% indicando uma melhora em relação à Regressão Logística, além disso vemos que apesar dos dados continuarem desbalanceados, vemos que o recall das classes para o teste possui valores aproximados.

In [ ]:
# Importância dos coeficientes
importances3 = clf3.feature_importances_
feature_importances3 = pd.DataFrame(
    importances3,
    columns = ['Importance'],
    index=df3.drop('Morte', axis=1).columns
).sort_values(by='Importance', ascending=False)

print("\nImportância das Features:")
print(feature_importances3)

* Ao verificar a importância das features podemos notar que novamente a Bilirrubina possui maior valor em relação as demais variaveis, portanto é mais utilizada para dividir os nós o que resulta na diminuição de impurezas.

In [ ]:
# Plottar a Árvore de Decisão
fig3 = plt.figure(figsize=(90,40))
_ = tree.plot_tree(clf3,
                      feature_names=df3.drop(columns=['Morte']).columns,
                      class_names=clf3.classes_,
                      filled=True,
                      fontsize=10,
                      )

plt.show()

In [ ]:
# Matriz de Confusão
matrixTree3 = confusion_matrix(y_test_tree3, tree_pred3)

plt.figure(figsize=(6, 6))

matrixTreePlot3 = ConfusionMatrixDisplay(confusion_matrix=matrixTree3, display_labels=clf3.classes_)
matrixTreePlot3.plot(cmap=plt.cm.Blues)

plt.title('Árvore de Decisão - Matriz de Confusão')
plt.xlabel('Prevista')
plt.ylabel('Real')

plt.show()

* Avaliando a matriz de confusão:
    * O modelo ainda classifica alguns casos erroneamente, mas como o recall possui valores aproximados entre as classes, vemos que há um equilibrio para os casos classificados erroneamente.

### Verificando os resultados da Árvore de Decisão

In [ ]:
y_test3[55]

In [ ]:
extract_rules(clf3, x_test3[55])


## **Reestruturação da Árvore de Decisão**

* Apesar da árvore de decisão entregar bons resultados, é possível melhorar o resultado através do GridSearchCV para encontrar os melhores parâmetros

In [ ]:
# Grid Search
path3 = clf3.cost_complexity_pruning_path(X_train_tree3, y_train_tree3)

param_grid3 = {
    'max_depth': range(2, 10),
    'min_samples_split': range(2, 10),
    'min_samples_leaf': range(1, 5),
    'criterion': ['gini', 'entropy'],
    'max_features': range(4, X3.shape[1] + 1),
    'ccp_alpha': (0.0, 0.1, 0.5, 0.05, 0.005)
}

CV_clf = GridSearchCV(estimator=clf3, param_grid=param_grid3, cv = 7, verbose=2, n_jobs=-1)
CV_clf.fit(X_train_tree3, y_train_tree3)

best_max_depth3 = CV_clf.best_estimator_.max_depth
best_min_split3 = CV_clf.best_estimator_.min_samples_split
best_min_leaf3 = CV_clf.best_estimator_.min_samples_leaf
best_criterion3 = CV_clf.best_estimator_.criterion
best_max_features3 = CV_clf.best_estimator_.max_features
best_ccp_alpha3 = CV_clf.best_estimator_.ccp_alpha

* Com os novos parâmentros, instanciamos uma nova árvore de decisão buscando melhores resultados.

In [ ]:
# Instanciar um novo objeto para a Árvore de Decisão Reestruturada
clf_ccp3 = DecisionTreeClassifier(criterion=best_criterion3,
                                   max_depth=6, # O resultado do GridSearch acaba tornando a árvore muito ilegível, então vamos manter 6 aqui
                                   min_samples_leaf=best_min_leaf3, 
                                   min_samples_split=best_min_split3, 
                                   ccp_alpha=best_ccp_alpha3,
                                   max_features=best_max_features3)

clf_ccp3.fit(X_train_tree3, y_train_tree3)

# Predict da Árvore de Decisão Reestruturada - Teste
tree_pred_ccp3 = clf_ccp3.predict(X_test_tree3)

# Predict da Árvore de Decisão Reestruturada - Treino
tree_pred_ccp3T = clf_ccp3.predict(X_train_tree3)

In [ ]:
# Tabela de Resultados
print("Report - Árvore de Decisão Reestruturada - Treino:")
print(classification_report(y_train_tree3, tree_pred_ccp3T, zero_division=0))


In [ ]:
# Tabela de Resultados
print("Report - Árvore de Decisão Reestruturada - Teste:")
print(classification_report(y_test_tree3, tree_pred_ccp3, zero_division=0))

* A nova árvore atingiu cerca de 81% de acurácia, isto se deve principalmente por conta das limitações de niveis que foram impostas. Porém agora possuimos uma árvore com mais interpretabilidade.

In [ ]:
# Importância dos coeficientes
importances_ccp3 = clf_ccp3.feature_importances_
feature_importances_ccp3 = pd.DataFrame(
    importances_ccp3, columns = ['Importance'], index=df3.drop('Morte', axis=1).columns).sort_values(by='Importance', ascending=False)

print("\nImportância das Features:")
print(feature_importances_ccp3)

* Com base na análise da importância das features, a bilirrubina(mg/dl) é a feature mais importante na classificação, seguida pela albumina(gm/dl) e pela aspartato aminotransferase(U/L).

In [ ]:
# Árvore de Decisão Reestruturada
fig_3 = plt.figure(figsize=(90,40))
_ = tree.plot_tree(clf_ccp3,
                      feature_names=df3.drop(columns=['Morte']).columns,
                      class_names=clf_ccp3.classes_,
                      filled=True,
                      fontsize=10,
                      )

plt.show()

In [ ]:
# Matriz de Confusão
matrixTree3T = confusion_matrix(y_test_tree3, tree_pred3)

plt.figure(figsize=(6, 6))

matrixTreePlot3 = ConfusionMatrixDisplay(confusion_matrix=matrixTree3T, display_labels=clf_ccp3.classes_)
matrixTreePlot3.plot(cmap=plt.cm.Blues)

plt.title('Árvore de Decisão Reestruturada - Matriz de Confusão')
plt.xlabel('Prevista')
plt.ylabel('Real')

plt.show()

* Apesar da reestruturação da árvore de decisão. é possível notar que a taxa de recall continua bem próxima da árvore anterior.

In [ ]:
# Salvando o modelo de árvore de decisão para uso futuro
print("\nO modelo de Árvore de Decisão Corrigida foi salvo em:")
print(dump(clf_ccp3, 'models/modelos_morte/death_tree_fixed3.joblib'))

## **Rede Neural**

* Buscando utilizar outro modelo para comparação, nesta seção utilizamos o modelo de Rede Neural. Este modelo necessita de que as classes não estejam em string, portanto é utilizado o *factorize* para transformar o valores string para binário.

In [ ]:
df3N = df.copy()

# Retira as linhas que possuem a coluna 'Situação' com o valor 'Transplante'
df3N = df3N[df3N['Situação'] != 'Transplante']

df3N['Situação'] = pd.factorize(df3N['Situação'])[0]

X3N  = df3N.drop(columns=['Situação']).values
y3N = df3N['Situação'].values

X_train3N, X_test3N, y_train3N, y_test3N = train_test_split(X3N, y3N, test_size=0.2)
X_train3N = scaler3.fit_transform(X_train3N)
X_test3N = scaler3.transform(X_test3N)

In [ ]:
neural3 = Sequential([
    Dense(32, activation='relu', input_shape=(x_train3.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='softmax')
])

* Criação da rede neural 3:

    * camada de entrada com 32 neurônios e função de ativação *relu*;

    * duas camadas com 64 neurônios;

    * uma camada com 32 neurônios;

    * camada de saída com 2 neurônios (para as duas classes).

In [ ]:
neural3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
neural3.summary()

* Abaixo teremos o treinamento da Rede Neural 3 com 200 épocas de 64 amostras.

In [ ]:
history3 = neural3.fit(X_train3N, y_train3N, epochs=200, batch_size=64, validation_data=(X_test3N, y_test3N))

* A acurácia começa a estabilizar entre 88%. Logo Abaixo temos os gráficos de acurácia e de perda:

In [ ]:
# Extract training and validation accuracy/loss
train_acc3 = history3.history['accuracy']
val_acc3 = history3.history['val_accuracy']
train_loss3 = history3.history['loss']
val_loss3 = history3.history['val_loss']

# Plot accuracy
plt.figure(figsize=(8, 5))
plt.plot(train_acc3, label='Training Accuracy')
plt.plot(val_acc3, label='Validation Accuracy')
plt.title('Accuracy Curves (Best Model)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

In [ ]:
# Plot loss (error)
plt.figure(figsize=(8, 5))
plt.plot(train_loss3, label='Training Loss')
plt.plot(val_loss3, label='Validation Loss')
plt.title('Loss Curves (Best Model)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

In [ ]:
# Tabela de report de Treino
print("Report - Dados de Treino:")
print(classification_report(y_train3N, np.argmax(neural3.predict(X_train3N), axis=1), zero_division=0))


In [ ]:
# Tabela de report de Teste
print("Report - Dados de Teste:")
print(classification_report(y_test3N, np.argmax(neural3.predict(X_test3N), axis=1), zero_division=0))

* O modelo apresenta um recall razoavelmente bom apesar de um desbalanceamento para a classe de sobreviveu, com uma acurácia geral de 88%.

In [ ]:
print("O modelo de Rede Neural foi salvo em:")
dump(neural3, 'models/modelos_morte/death_neural3.joblib')

## **Light Gradient Boosting Machine (LGBM)**

* Nesta seção utilizaremos o **Light Gradient Boosting Machine** (ou LGBM). O qual faz um processo de ensemble de tipo boosting, que vai combinar vários modelos do mesmo algoritmo de forma sequencial. Assim como a Rede Neural, utiliza os dados em binário.

In [ ]:
# Cria os datasets para LightGBM
train_data3 = lgb.Dataset(X_train3N, label=y_train3N)
valid_data3 = lgb.Dataset(X_test3N, label= y_test3N)

# Define os parâmetros do modelo
parameters3 = {
    'objective': 'multiclass',  # Define o objetivo como classificação multiclasse
    'boosting_type': 'gbdt',  # Usa Gradient Boosting Decision Tree
    'metric': 'multi_logloss',  # Métrica de avaliação: log-loss para múltiplas classes
    'num_class': len(np.unique(y3N)),  # Número de classes únicas em 'Situação'
    'num_leaves': 31,  # Número máximo de folhas em cada árvore
    'learning_rate': 0.05,  # Taxa de aprendizado do modelo
    'feature_fraction': 0.9  # Fração de features usadas em cada iteração de construção da árvore
}

# Treina o modelo
model_lgbm3 = lgb.train(parameters3, train_data3, valid_sets=[train_data3, valid_data3], num_boost_round=5000)


In [ ]:
# Faz previsões
y_train_pred_3 = model_lgbm3.predict(X_train3N)
y_valid_pred_3 = model_lgbm3.predict(X_test3N)

# Converte as previsões para rótulos de classe
y_train_pred_3_labels = np.argmax(y_train_pred_3, axis=1)
y_valid_pred_3_labels = np.argmax(y_valid_pred_3, axis=1)


In [ ]:
# Avalia o modelo no conjunto de treino
print("Report - LGBM - Treino:")
print(classification_report(y_train3N, y_train_pred_3_labels, zero_division=0))

In [ ]:

# Avalia o modelo
print("Report - LGBM - Teste:")
print(classification_report(y_test3N, y_valid_pred_3_labels, zero_division=0))

* A acurácia geral do modelo no conjunto de treino é de 100%, mas apesar disso, na tabela de teste vemos que o modelo possui uma acurácia de 98% isso siginifica que o modelo se adaptou bem ao conjunto de treino,

In [ ]:
# Matriz de Confusão
matrixLGBM3T = confusion_matrix(y_test3N, y_valid_pred_3_labels)

plt.figure(figsize=(6, 6))
matrixLGBMPlot3 = ConfusionMatrixDisplay(confusion_matrix=matrixLGBM3T, display_labels=np.unique(y3N))
matrixLGBMPlot3.plot(cmap=plt.cm.Blues)

plt.title('LightGBM - Matriz de Confusão')
plt.xlabel('Prevista')
plt.ylabel('Real')

plt.show()

* Analisando a matriz de confusão vemos que em relação aos modelos anteriores o LGMB é o que consegue classificar melhor, apesar de um leve desbalanceamento entre as classes que não altera a eficácia do modelo.

In [ ]:
print("O modelo de LightGBM foi salvo em:")
dump(model_lgbm3, 'models/modelos_morte/death_lightgbm3.joblib')

## **Comparando os Modelos**

* Abaixo teremos os reports dos testes de cada modelo utilizado:

In [ ]:
# Tabela de Resultados - Teste - Regressão Logística
print("Report - Regressão Logistica:")
print(classification_report(y_test3, y_pred3, zero_division=0))


In [ ]:
# Tabela de Resultados - Teste - Árvore de Decisão
print("Report - Árvore de Decisão Reestruturada:")
print(classification_report(y_test_tree3, tree_pred_ccp3, zero_division=0))

In [ ]:
# Tabela de report de Teste - Rede Neural
print("Report - Rede Neural:")
print(classification_report(y_test3N, np.argmax(neural3.predict(X_test3N), axis=1), zero_division=0))

In [ ]:

# Tabela de report de Teste - LGBM
print("Report - LGBM:")
print(classification_report(y_test3N, y_valid_pred_3_labels, zero_division=0))

* A seguir as features de Rede Neural e da Ávore de Decisão Reestruturada

In [ ]:
# Importância dos coeficientes - Regressão Logística
feature_names3 = df3.columns[1:]
coefficients3 = logreg3.coef_
coef_df3 = pd.DataFrame(coefficients3, columns=feature_names3)
for idx, class_name in enumerate(logreg3.classes_):
    sorted_coef3 = coef_df3.iloc[idx].sort_values(ascending=False)
    print(f"Importância das Features em relação a {class_name} na Regressão Logistica:\n")
    print(sorted_coef3)
    break

In [ ]:
# Importância dos coeficientes - Árvore de Decisão
importances_ccp3 = clf_ccp3.feature_importances_
feature_importances_ccp3 = pd.DataFrame(
    importances_ccp3, columns = ['Importance'], index=df3.drop('Morte', axis=1).columns).sort_values(by='Importance', ascending=False)

print("\nImportância das Features:")
print(feature_importances_ccp3)

* Avaliação dos Modelos

    Desempenho dos Modelos
    Ao comparar os resultados dos modelos utilizados, os dois modelos com maior desempenho são o LGBM e a Rede Neural, nesta ordem. Isso se deve a dois fatores principais:

    * Rede Neural: Proporcionou um balanceamento do recall entre as classes "Sobreviveu" e "Morte", mesmo com a diferença de volume de dados entre elas.
    
    * LGBM: Apresentou uma alta acurácia.

* Importância das Variáveis

    Entre as variáveis utilizadas para construir as classificações, nos modelos de Regressão Logística e Árvore de Decisão, a Bilirrubina se destaca como o atributo mais importante:

    * Regressão Logística:
        A Bilirrubina possui uma correlação forte e inversa com a variável "Morte". Isso significa que níveis mais altos de bilirrubina estão associados a um maior risco de morte.
        
    * Árvore de Decisão:
        A Bilirrubina é responsável por pelo menos 50% da redução de impurezas na árvore de decisão. Isso indica que essa variável é frequentemente usada para dividir os nós e melhorar a pureza da árvore.
        A importância da Bilirrubina se deve ao fato de este atributo estar relacionado com a perda da função hepática, o que está associado a um maior risco de morte.

---
# Conclusão
---